In [16]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [17]:
#path = '../toxicdata17/'
#comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE='glove.6B.50d.txt'
#TRAIN_DATA_FILE=f'{path}{comp}train.csv'
TEST_DATA_FILE='test.csv'
TRAIN_DATA_FILE=pd.read_csv("train.csv")

In [18]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [19]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

list_sentences_train = train["posts"]
list_classes = ["is_E", "is_S" , "is_T","is_J"]
y = train[list_classes].values
list_sentences_test = test["posts"]

In [20]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [21]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [22]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


(0.020940498, 0.6441043)

In [23]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [24]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(4, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

/Users/nalisha/opt/anaconda3/envs/nash/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 5464 samples, validate on 608 samples
Epoch 1/2
5464/5464 [==============================] - 63s 12ms/step - loss: 0.5808 - accuracy: 0.6913 - val_loss: 0.5661 - val_accuracy: 0.7052
Epoch 2/2
5464/5464 [==============================] - 56s 10ms/step - loss: 0.5651 - accuracy: 0.7130 - val_loss: 0.5502 - val_accuracy: 0.7286


In [31]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('test.csv')

2603/2603 [==============================] - 1s 532us/step


In [32]:
sample_submission[["is_E", "is_S" , "is_T","is_J"]] = y_test
sample_submission.to_csv('sample_submission_lstm.csv', index=False)